# Creating Dataset for Number of Speaker Estimation

This notebook shows how we have created the dataset for the Number of Speaker Estimation project.

We first start with importing `util.py`, where all necessary code is provided.

In [1]:
import sys
sys.path.insert(0, './src/')
from src import util

dataset = "train100"

Important to note is that for this code to work, LibriSpeech has to be downloaded and unzipped in a `./data/` folder. The general folder path is then:
```
./data/LibriSpeech/type_of_librispeech/speaker_id/chapter_id/audiofile.flac
```
If you have downloaded, for example, the LibriSpeech train-100 clean dataset, the following path would be valid:
```
./data/LibriSpeech/train-clean-100/19/198/19-198-0000.flac
```

**Please note that running this notebook requires a lot of harddisk space. At each step, we save the output as new files in a new folder. This is intentional, for testing purposes and to have a back-up of the previous steps. If you want to run this notebook withouth doing so, code in the source files has to be changed, or intermediate results should manually be removed from the harddisk.**

## 1. Rewrite LibriSpeech `.flac` to `.wav`

We first rewrite LibriSpeech to wav files, which we do by calling `util.flacs_to_wavs`.
This function requires two parameters:

 - `data_dir` represents where the original data is located. According to the beforementioned structure, this would be `./data/LibriSpeech/'
 - `new_dir` represents the new location where the `.wav` files will be written to.

In the cell below, the `new_dir` is called wavs100, to indicate the file format (`.wav`) and to remind that this is from train-100 clean.

In [3]:
data_dir = "./data/{}/".format(dataset)
new_dir = "./data/{}wavs/".format(dataset)

In [4]:
util.flacs_to_wavs(data_dir, new_dir)

331it [01:14,  4.44it/s]


## 2. Split `.wav` files in fragments of 5 seconds

The function `util.split_audio_in_samples` is written to do this. It requires three parameters:

 - `data_dir` represents where the original `.wav` files are located. According to the previous cell with code, this is `./data/wavs100/`
 - `new_dir` represents the new location where the shorter `.wav` files will be written to.
 - `t` is how long the files should be in seconds, in our case `t = 5`.

In [5]:
data_dir = "./data/{}wavs/".format(dataset)
new_dir = "./data/{}splits_test/".format(dataset)
t = 10

In [6]:
util.create_audio_splits(data_dir, new_dir, t)

331it [00:44,  7.43it/s]


## 3. Change Loudness of All Splits to 70 dB

The function `util.change_loudness` will change the loudness of every split to 70 dB. It requires three parameters:
 - `data_dir` denotes where the splitted `.wav` files are located.
 - `new_dir` denotes where the new files will be written to.
 - `target` denotes the target level of decibels. Default is set to 70.
 
Since we use a subprocess call in this function, this function will run a bit longer than the previous function calls.

In [4]:
data_dir = "./data/{}splits_test/".format(dataset)
new_dir = "./data/{}normalized/".format(dataset)
target = 70.

In [ ]:
util.change_loudness(data_dir, new_dir, target)

25it [03:54, 10.68s/it]

## 4. Merging Files

For merging splits, and hence actually creating the datasets, we use the function `util.merge_splits`. It takes three parameters:
 - `data_dir` represents where the normalized split files are located.
 - `new_dir` represents the location where the merged split files will be written to
 - `max_number_of_speakers`

This function is different from the others before, since this function creates another folder in `new_dir`, namely a folder named `metadata`, where `.txt` files are saved containing the speaker IDs from the speakers that are merged. The name of this `.txt` file corresponds with the merged `.wav` file.

In [9]:
data_dir = "./data/{}normalized/".format(dataset)
new_dir = "./data/trainset100/"
max_nr_of_speakers = 10

In [10]:
util.merge_audiofiles(data_dir, new_dir, max_nr_of_speakers)

/Users/jordynaus/Documents/Artificial Intelligence/Master/2021 Sem. 2/Automatic Speech Recognition/NumberOfSpeakerEstimation/src/util.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  files_per_speaker = np.array(files_per_speaker)
0it [00:00, ?it/s]

11810 files found in total
Now starting to merge files...


100it [00:54,  1.85it/s]

Created 4854 unique datapoints


## 5. Creating STFTs from Audio Data

In [11]:
data_dir = "./data/trainset100/train"
new_dir = "./data/trainset100/stft"

In [12]:
util.create_spectrograms(data_dir, new_dir)

100%|██████████| 441/441 [00:26<00:00, 16.78it/s]
12it [04:20, 21.69s/it]


## 6. Done

Now we have generated our dataset, which we can use in the Number of Speaker Estimation task. 
In summary, we did the following:
 - Rewrite `.flac` to `.wav`
 - Split `.wav` files in fragments of 5 seconds
 - Normalized loudness in every file to be 70 dB
 - Merged files together, taking into account that:
     - Two speakers can never be in the same merged file;
     - Classes are as balanced as possible, although perfect class balance is difficult to achieve;
     - Normalized merged data arrays to be in the range of [-1, 1] to prevent clipping.
 - Created STFTs from the audio files, such that we can directly pass the data to the network.